<a href="https://colab.research.google.com/github/shubham8garg/MCP_A2A_session/blob/main/A2A-updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 A2A (Agent-to-Agent Protocol) — Live Demo

**Prerequisite:** Run the MCP demo notebook first (or understand MCP basics).

This notebook has **two sections**:

| Section | What | Who |
|---------|------|-----|
| **🎯 SECTION A: See the Magic** | Run agents, watch them collaborate | Everyone |
| **🔬 SECTION B: Under the Hood** | Agent code, response parsing, architecture | Curious minds |

**Instructions:** Just run each cell top to bottom. Section A takes ~5 minutes.

In [ ]:
!pip install openai mcp fastmcp httpx python-dotenv a2a-sdk uvicorn httpx pyngrok -q

In [ ]:
import os

# ── API Keys & Auth Tokens (set these before running) ──
os.environ["OPENAI_API_KEY"] = "sk-abc"

# Get your ngrok token at: https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTHTOKEN"  # <---- REPLACE WITH YOUR TOKEN!

assert os.environ.get("OPENAI_API_KEY", "").startswith("sk-"), "❌ Please set a valid OPENAI_API_KEY"
print("✅ OPENAI_API_KEY is set")
if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTHTOKEN":
    print("⚠️  NGROK_AUTH_TOKEN not set — ngrok tunnels will not work (needed only for A2A Inspector)")
else:
    print("✅ NGROK_AUTH_TOKEN is set")

In [ ]:
# ── Setup: imports, helpers, agent files, MCP server file ──
import subprocess, sys, os, signal, time, json, asyncio, uuid
import httpx
from openai import OpenAI
from a2a.types import (
    AgentCard, AgentSkill, AgentCapabilities,
    Message, TextPart, DataPart, Part, Task, TaskState, TaskStatus, Artifact,
    SendMessageRequest, SendMessageResponse, MessageSendParams,
)
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import InMemoryTaskStore
from a2a.client import A2AClient, A2ACardResolver
from mcp.client.stdio import stdio_client, StdioServerParameters
from mcp.client.session import ClientSession

client_openai = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
MODEL = "gpt-5.2"

def print_header(text):
    width = 60
    print("\n" + "═" * width)
    print(f"  {text}")
    print("═" * width)

def print_step(num, text):
    print(f"\n▶ Step {num}: {text}")
    print("─" * 55)

def print_tool_call(name, args):
    print(f"  📤 TOOL CALL: {name}")
    if isinstance(args, str): args = json.loads(args)
    for k, v in args.items():
        print(f"     {k}: {v}")

def print_tool_result(text, max_lines=8):
    lines = text.split("\n")
    for line in lines[:max_lines]:
        print(f"  📥 {line}")
    if len(lines) > max_lines:
        print(f"  📥 ... ({len(lines) - max_lines} more lines)")

AGENT_PROCESSES = {}

def start_agent_server(name, script_path, port):
    stop_agent_server(name)
    abs_path = os.path.abspath(script_path)
    proc = subprocess.Popen(
        [sys.executable, abs_path],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        env={**os.environ, "PYTHONPATH": os.pathsep.join(sys.path)},
    )
    time.sleep(3)
    if proc.poll() is not None:
        _, stderr = proc.communicate()
        print(f"❌ {name} crashed on startup!")
        print(stderr.decode()[-500:])
        return False
    AGENT_PROCESSES[name] = proc
    print(f"✅ {name} running on port {port} (PID: {proc.pid})")
    return True

def stop_agent_server(name):
    if name in AGENT_PROCESSES:
        proc = AGENT_PROCESSES[name]
        try:
            proc.terminate()
            proc.wait(timeout=5)
        except:
            proc.kill()
        del AGENT_PROCESSES[name]
        print(f"🛑 {name} stopped")

def stop_all_agents():
    for name in list(AGENT_PROCESSES.keys()):
        stop_agent_server(name)
    print("🛑 All agents stopped")

def extract_text_from_message(message):
    texts = []
    if message and hasattr(message, 'parts') and message.parts:
        for part in message.parts:
            if hasattr(part, 'root') and hasattr(part.root, 'text'):
                texts.append(part.root.text)
            elif hasattr(part, 'text'):
                texts.append(part.text)
    return "\n".join(texts)

def extract_text_from_response(resp):
    texts = []
    try:
        result_obj = resp.root.result
        if hasattr(result_obj, 'history') and result_obj.history:
            for msg in result_obj.history:
                if hasattr(msg, 'role') and str(msg.role) != 'user':
                    text = extract_text_from_message(msg)
                    if text: texts.append(text)
        if hasattr(result_obj, 'artifacts') and result_obj.artifacts:
            for art in result_obj.artifacts:
                if hasattr(art, 'parts'):
                    for part in art.parts:
                        if hasattr(part, 'root') and hasattr(part.root, 'text'):
                            texts.append(part.root.text)
        if hasattr(result_obj, 'parts'):
            text = extract_text_from_message(result_obj)
            if text: texts.append(text)
        if hasattr(result_obj, 'status') and result_obj.status:
            if hasattr(result_obj.status, 'message') and result_obj.status.message:
                status_text = extract_text_from_message(result_obj.status.message)
                if status_text and status_text not in texts:
                    texts.append(status_text)
    except Exception as e:
        texts.append(f"(Parse error: {e})")
    return "\n".join(texts) if texts else str(resp)[:300]

async def send_task_to_agent(base_url, user_text):
    async with httpx.AsyncClient(timeout=30.0) as httpx_client:
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=base_url)
        card = await resolver.get_agent_card()
        a2a_client = A2AClient(httpx_client=httpx_client, agent_card=card)
        message = Message(
            message_id=str(uuid.uuid4()), role="user",
            parts=[Part(root=TextPart(text=user_text))],
        )
        params = MessageSendParams(message=message)
        request = SendMessageRequest(id=str(uuid.uuid4()), params=params)
        print(f'  📤 Sending: "{user_text}"')
        response = await a2a_client.send_message(request=request)
        print(f"  📥 Response received!")
        return response

async def discover_agent(base_url):
    async with httpx.AsyncClient() as httpx_client:
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=base_url)
        card = await resolver.get_agent_card()
        print(f"📇 Agent Card Discovered!\n")
        print(f"   Name:        {card.name}")
        print(f"   Description: {card.description}")
        print(f"   URL:         {card.url}")
        print(f"   Version:     {card.version}")
        print(f"\n   🎯 Skills ({len(card.skills)}):")
        for skill in card.skills:
            print(f"      • {skill.name}: {skill.description}")
            if skill.examples:
                print(f"        Examples: {skill.examples}")
        return card

def mcp_tools_to_openai_functions(mcp_tools):
    openai_tools = []
    for tool in mcp_tools:
        openai_tools.append({
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description or "",
                "parameters": tool.inputSchema if tool.inputSchema else {"type": "object", "properties": {}},
            },
        })
    return openai_tools

# ── Create all files (handles Colab session expiry) ──
os.makedirs('a2a_agents', exist_ok=True)
os.makedirs('mcp_server', exist_ok=True)

# Write currency agent
_CURRENCY_AGENT_CODE = r'''"""
A2A Agent: Currency Converter
"""

import json
import re
import uuid
import uvicorn

from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCard, AgentSkill, AgentCapabilities,
    Message, TextPart, Part,
)


EXCHANGE_RATES = {
    "USD": 1.0, "JPY": 149.50, "EUR": 0.92, "GBP": 0.79,
    "INR": 83.50, "AUD": 1.53, "CAD": 1.36,
}


def convert_currency(amount, from_curr, to_curr):
    from_curr = from_curr.upper()
    to_curr = to_curr.upper()
    if from_curr not in EXCHANGE_RATES or to_curr not in EXCHANGE_RATES:
        return None, f"Unknown currency. Supported: {list(EXCHANGE_RATES.keys())}"
    usd_amount = amount / EXCHANGE_RATES[from_curr]
    result = usd_amount * EXCHANGE_RATES[to_curr]
    return result, None


def parse_conversion_request(text):
    text = text.upper().replace(",", "")
    pattern = r"(\d+(?:\.\d+)?)\s*([A-Z]{3})\s*(?:TO|->|=>|IN)\s*([A-Z]{3})"
    match = re.search(pattern, text)
    if match:
        return float(match.group(1)), match.group(2), match.group(3)
    return None, None, None


class CurrencyAgentExecutor(AgentExecutor):

    async def execute(self, context: RequestContext, event_queue: EventQueue):
        # Extract text from message parts
        user_text = ""
        if context.message and context.message.parts:
            for part in context.message.parts:
                if hasattr(part, "root") and hasattr(part.root, "text"):
                    user_text += part.root.text
                elif hasattr(part, "text"):
                    user_text += part.text

        # Parse and convert
        amount, from_curr, to_curr = parse_conversion_request(user_text)

        if amount is None:
            response_text = (
                "I can convert currencies! Try: '100 USD to JPY'\n"
                f"Supported currencies: {', '.join(EXCHANGE_RATES.keys())}"
            )
        else:
            result, error = convert_currency(amount, from_curr, to_curr)
            if error:
                response_text = f"Error: {error}"
            else:
                rate = round(EXCHANGE_RATES[to_curr] / EXCHANGE_RATES[from_curr], 4)
                response_text = (
                    f"{amount:,.2f} {from_curr} = {result:,.2f} {to_curr}\n"
                    f"(Rate: 1 {from_curr} = {rate} {to_curr})"
                )

        # ⭐ THE FIX: await the enqueue_event call!
        await event_queue.enqueue_event(
            Message(
                message_id=str(uuid.uuid4()),
                role="agent",
                parts=[Part(root=TextPart(text=response_text))],
            )
        )

    async def cancel(self, context: RequestContext, event_queue: EventQueue):
        pass


agent_card = AgentCard(
    name="Currency Converter Agent",
    description="Converts between currencies using real-time exchange rates.",
    url="http://localhost:5001",
    version="1.0.0",
    defaultInputModes=["text"],
    defaultOutputModes=["text"],
    skills=[
        AgentSkill(
            id="currency-conversion",
            name="Currency Conversion",
            description="Convert an amount from one currency to another. Example: '100 USD to JPY'",
            tags=["currency", "finance", "conversion"],
            examples=["100 USD to JPY", "500 EUR to GBP", "1000 INR to USD"],
        )
    ],
    capabilities=AgentCapabilities(streaming=False),
)

executor = CurrencyAgentExecutor()
task_store = InMemoryTaskStore()
handler = DefaultRequestHandler(agent_executor=executor, task_store=task_store)
app = A2AStarletteApplication(agent_card=agent_card, http_handler=handler)

if __name__ == "__main__":
    print("Currency Converter Agent starting on port 5001...")
    uvicorn.run(app.build(), host="0.0.0.0", port=5001)'''

with open('a2a_agents/currency_agent.py', 'w') as f:
    f.write(_CURRENCY_AGENT_CODE)

# Write finance agent
_FINANCE_AGENT_CODE = r'''"""
A2A Agent: Finance Advisor
Uses LLM (OpenAI) internally to analyze budget requests.
"""

import os
import json
import uuid
import uvicorn
from openai import OpenAI

from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCard, AgentSkill, AgentCapabilities,
    Message, TextPart, Part,
)


EXCHANGE_RATES = {"USD": 1.0, "JPY": 149.50, "EUR": 0.92, "GBP": 0.79}

COST_OF_LIVING = {
    "Tokyo": {"food": 60, "transport": 15, "activities": 40, "misc": 20},
    "Paris": {"food": 70, "transport": 12, "activities": 50, "misc": 25},
    "London": {"food": 65, "transport": 18, "activities": 45, "misc": 22},
    "New York": {"food": 55, "transport": 15, "activities": 35, "misc": 20},
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_daily_cost_estimate",
            "description": "Get estimated daily costs for a city",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "convert_currency",
            "description": "Convert an amount between currencies",
            "parameters": {
                "type": "object",
                "properties": {
                    "amount": {"type": "number"},
                    "from_currency": {"type": "string"},
                    "to_currency": {"type": "string"},
                },
                "required": ["amount", "from_currency", "to_currency"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_budget_feasibility",
            "description": "Check if planned expenses fit within a budget",
            "parameters": {
                "type": "object",
                "properties": {
                    "total_budget": {"type": "number"},
                    "flight_cost": {"type": "number"},
                    "hotel_cost": {"type": "number"},
                    "num_days": {"type": "integer"},
                    "city": {"type": "string"},
                },
                "required": ["total_budget"],
            },
        },
    },
]


def execute_tool(name, args):
    if name == "get_daily_cost_estimate":
        city = args.get("city", "").title()
        costs = COST_OF_LIVING.get(city)
        if not costs:
            return json.dumps({"error": f"No data for {city}", "available": list(COST_OF_LIVING.keys())})
        costs_copy = dict(costs)
        costs_copy["total_per_day"] = sum(costs_copy.values())
        return json.dumps({"city": city, "daily_costs_usd": costs_copy})

    elif name == "convert_currency":
        amount = args["amount"]
        from_c = args["from_currency"].upper()
        to_c = args["to_currency"].upper()
        if from_c not in EXCHANGE_RATES or to_c not in EXCHANGE_RATES:
            return json.dumps({"error": "Unknown currency"})
        result = (amount / EXCHANGE_RATES[from_c]) * EXCHANGE_RATES[to_c]
        return json.dumps({"from": from_c, "to": to_c, "amount": amount, "result": round(result, 2)})

    elif name == "analyze_budget_feasibility":
        budget = args["total_budget"]
        flight = args.get("flight_cost", 0)
        hotel = args.get("hotel_cost", 0)
        days = args.get("num_days", 5)
        city = args.get("city", "Tokyo").title()
        costs = COST_OF_LIVING.get(city, {"food": 50, "transport": 15, "activities": 35, "misc": 15})
        daily = sum(costs.values())
        total = flight + hotel + (daily * days)
        remaining = budget - total
        return json.dumps({
            "budget": budget, "total_estimated": round(total, 2),
            "remaining": round(remaining, 2), "feasible": remaining >= 0,
            "breakdown": {"flights": flight, "hotel": hotel, "daily_expenses": daily * days},
            "recommendation": "Within budget!" if remaining >= 0 else f"Over budget by ${abs(remaining):.0f}",
        })

    return json.dumps({"error": f"Unknown tool: {name}"})


def run_llm_with_tools(user_message):
    openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    messages = [
        {
            "role": "system",
            "content": (
                "You are a financial advisor agent specializing in travel budgets. "
                "Use your tools to provide accurate budget analysis. "
                "Always use tools for data. Be concise and structured."
            ),
        },
        {"role": "user", "content": user_message},
    ]

    for _ in range(5):
        response = openai_client.chat.completions.create(
            model="gpt-5.2", messages=messages, tools=TOOLS, temperature=0,
        )
        choice = response.choices[0]

        if choice.finish_reason == "tool_calls" and choice.message.tool_calls:
            messages.append(choice.message.model_dump())
            for tc in choice.message.tool_calls:
                result = execute_tool(tc.function.name, json.loads(tc.function.arguments))
                messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})
        elif choice.finish_reason == "stop":
            return choice.message.content
        else:
            return choice.message.content or "Unable to process request."

    return "Max iterations reached."


class FinanceAgentExecutor(AgentExecutor):

    async def execute(self, context: RequestContext, event_queue: EventQueue):
        user_text = ""
        if context.message and context.message.parts:
            for part in context.message.parts:
                if hasattr(part, "root") and hasattr(part.root, "text"):
                    user_text += part.root.text
                elif hasattr(part, "text"):
                    user_text += part.text

        if not user_text.strip():
            user_text = "What can you help with?"

        result = run_llm_with_tools(user_text)

        # ⭐ THE FIX: await the enqueue_event call!
        await event_queue.enqueue_event(
            Message(
                message_id=str(uuid.uuid4()),
                role="agent",
                parts=[Part(root=TextPart(text=result))],
            )
        )

    async def cancel(self, context: RequestContext, event_queue: EventQueue):
        pass


agent_card = AgentCard(
    name="Finance Advisor Agent",
    description="Analyzes travel budgets, estimates costs, converts currencies, and checks financial feasibility using AI.",
    url="http://localhost:5002",
    version="1.0.0",
    defaultInputModes=["text"],
    defaultOutputModes=["text"],
    skills=[
        AgentSkill(
            id="budget-analysis",
            name="Budget Feasibility Analysis",
            description="Analyze if a travel plan fits within a given budget.",
            tags=["finance", "budget", "travel"],
            examples=[
                "Is a 5-day Tokyo trip feasible for $3000 with $820 flights and $950 hotel?",
                "Analyze budget: $2000 for 3 days in Paris",
            ],
        ),
        AgentSkill(
            id="cost-estimation",
            name="Daily Cost Estimation",
            description="Estimate daily expenses for a destination city.",
            tags=["finance", "costs"],
            examples=["What are daily costs in Tokyo?"],
        ),
        AgentSkill(
            id="currency-conversion",
            name="Currency Conversion",
            description="Convert amounts between currencies.",
            tags=["finance", "currency"],
            examples=["Convert 3000 USD to JPY"],
        ),
    ],
    capabilities=AgentCapabilities(streaming=False),
)

executor = FinanceAgentExecutor()
task_store = InMemoryTaskStore()
handler = DefaultRequestHandler(agent_executor=executor, task_store=task_store)
app = A2AStarletteApplication(agent_card=agent_card, http_handler=handler)

if __name__ == "__main__":
    print("Finance Advisor Agent starting on port 5002...")
    uvicorn.run(app.build(), host="0.0.0.0", port=5002)'''

with open('a2a_agents/finance_agent.py', 'w') as f:
    f.write(_FINANCE_AGENT_CODE)

# Write MCP travel server
_MCP_SERVER_CODE = r'''"""
Travel Planning MCP Server — Built with FastMCP 2.x
"""

import json
import sys
from datetime import datetime

try:
    from fastmcp import FastMCP
except ImportError as e:
    print(f"Import error: {e}", file=sys.stderr)
    sys.exit(1)

# ═══════════════════════════════════════════
# Create the MCP Server (FastMCP 2.x API)
# ═══════════════════════════════════════════
mcp = FastMCP("Travel Planning Tools")

# ═══════════════════════════════════════════
# SIMULATED DATA
# ═══════════════════════════════════════════

FLIGHT_DATABASE = {
    ("SFO", "TYO"): [
        {"airline": "ANA", "flight": "NH7", "depart": "11:30 AM", "arrive": "3:30 PM +1",
         "duration": "11h 00m", "price": 820, "stops": 0, "window_seat": True},
        {"airline": "JAL", "flight": "JL1", "depart": "1:00 PM", "arrive": "5:15 PM +1",
         "duration": "11h 15m", "price": 910, "stops": 0, "window_seat": True},
        {"airline": "United", "flight": "UA837", "depart": "2:45 PM", "arrive": "7:00 PM +1",
         "duration": "11h 15m", "price": 750, "stops": 0, "window_seat": False},
        {"airline": "Delta (via Seoul)", "flight": "DL295", "depart": "9:00 AM", "arrive": "8:30 PM +1",
         "duration": "17h 30m", "price": 620, "stops": 1, "window_seat": True},
    ],
    ("TYO", "SFO"): [
        {"airline": "ANA", "flight": "NH8", "depart": "5:00 PM", "arrive": "10:30 AM same day",
         "duration": "9h 30m", "price": 840, "stops": 0, "window_seat": True},
        {"airline": "JAL", "flight": "JL2", "depart": "6:30 PM", "arrive": "11:45 AM same day",
         "duration": "9h 15m", "price": 890, "stops": 0, "window_seat": True},
        {"airline": "United", "flight": "UA838", "depart": "4:00 PM", "arrive": "9:30 AM same day",
         "duration": "9h 30m", "price": 770, "stops": 0, "window_seat": True},
    ],
}

HOTEL_DATABASE = {
    "Tokyo": [
        {"name": "K5 Tokyo", "area": "Nihonbashi", "type": "boutique",
         "rating": 4.8, "price_night": 190, "vegetarian": True},
        {"name": "Trunk Hotel", "area": "Shibuya", "type": "boutique",
         "rating": 4.6, "price_night": 220, "vegetarian": True},
        {"name": "MUJI Hotel Ginza", "area": "Ginza", "type": "boutique",
         "rating": 4.5, "price_night": 175, "vegetarian": True},
        {"name": "APA Hotel Shinjuku", "area": "Shinjuku", "type": "business",
         "rating": 3.8, "price_night": 95, "vegetarian": False},
        {"name": "Park Hyatt Tokyo", "area": "Shinjuku", "type": "luxury",
         "rating": 4.9, "price_night": 550, "vegetarian": True},
    ]
}

CALENDAR = [
    {"date": "2026-10-01", "time": "09:00-10:00", "event": "Team standup", "flexible": True},
    {"date": "2026-10-03", "time": "14:00-15:00", "event": "Dentist appointment", "flexible": False},
    {"date": "2026-10-07", "time": "11:00-12:00", "event": "Project review", "flexible": False},
]

USER_PREFS = {
    "name": "Alex Johnson", "home_airport": "SFO",
    "seat": "window", "meal": "vegetarian", "hotel_type": "boutique",
    "budget_style": "moderate",
    "loyalty": ["ANA Mileage Club", "Marriott Bonvoy"],
    "interests": ["Japanese cuisine", "temples", "street photography", "local markets"],
    "past_trips": ["Kyoto 2023", "Osaka 2024"],
}

TOKYO_TIPS = {
    "best_months": ["March-April (Cherry Blossom)", "October-November (Autumn)"],
    "must_visit": ["Senso-ji Temple", "Meiji Shrine", "Tsukiji Market", "Shibuya Crossing"],
    "vegetarian_spots": ["Ain Soph Ripple (Shinjuku)", "T's TanTan (Tokyo Station)", "Bon (Taito)"],
    "tip": "Get a 72-hour Tokyo Metro Pass for 1,500 yen (~$10)",
}


# ═══════════════════════════════════════════
# TOOLS
# ═══════════════════════════════════════════

@mcp.tool()
def search_flights(origin: str, destination: str, date: str) -> str:
    """Search flights between two airports on a specific date. Returns real pricing and availability."""
    origin = origin.upper().strip()
    destination = destination.upper().strip()
    code_map = {"TOKYO": "TYO", "NRT": "TYO", "HND": "TYO", "SAN FRANCISCO": "SFO"}
    origin = code_map.get(origin, origin)
    destination = code_map.get(destination, destination)

    flights = FLIGHT_DATABASE.get((origin, destination), [])
    results = [dict(f, date=date) for f in flights]

    return json.dumps({
        "route": f"{origin} -> {destination}", "date": date,
        "count": len(results), "flights": results, "currency": "USD"
    }, indent=2)


@mcp.tool()
def search_hotels(city: str, checkin: str, checkout: str, hotel_type: str = "any") -> str:
    """Search hotels in a city with pricing. Filter by type: boutique, business, luxury, or any."""
    hotels = HOTEL_DATABASE.get(city.strip().title(), [])

    if hotel_type != "any":
        hotels = [h for h in hotels if h["type"] == hotel_type.lower()]

    try:
        nights = (datetime.strptime(checkout, "%Y-%m-%d") - datetime.strptime(checkin, "%Y-%m-%d")).days
    except ValueError:
        nights = 5

    results = [dict(h, checkin=checkin, checkout=checkout, nights=nights,
                    total_price=h["price_night"] * nights) for h in hotels]
    results.sort(key=lambda x: x["rating"], reverse=True)

    return json.dumps({
        "city": city.title(), "checkin": checkin, "checkout": checkout,
        "nights": nights, "count": len(results), "hotels": results, "currency": "USD"
    }, indent=2)


@mcp.tool()
def check_calendar(start_date: str, end_date: str) -> str:
    """Check user calendar for events and conflicts in a date range."""
    try:
        d1 = datetime.strptime(start_date, "%Y-%m-%d")
        d2 = datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        return json.dumps({"error": "Invalid date format. Use YYYY-MM-DD"})

    events_in_range = []
    conflicts = []
    for ev in CALENDAR:
        ev_date = datetime.strptime(ev["date"], "%Y-%m-%d")
        if d1 <= ev_date <= d2:
            events_in_range.append(ev)
            if not ev["flexible"]:
                conflicts.append(ev)

    return json.dumps({
        "range": f"{start_date} to {end_date}",
        "events": events_in_range, "conflicts": conflicts,
        "conflict_count": len(conflicts),
    }, indent=2)


@mcp.tool()
def estimate_daily_expenses(city: str, style: str = "moderate") -> str:
    """Estimate daily expenses for a city. Style: budget, moderate, or luxury."""
    costs = {
        "Tokyo": {
            "budget": {"food": 30, "transport": 10, "activities": 20, "misc": 10},
            "moderate": {"food": 60, "transport": 15, "activities": 40, "misc": 20},
            "luxury": {"food": 150, "transport": 50, "activities": 100, "misc": 50},
        }
    }
    city_costs = costs.get(city.title(), {}).get(style.lower())
    if not city_costs:
        return json.dumps({"error": f"No data for {city}/{style}"})

    city_costs["total_per_day"] = sum(city_costs.values())
    return json.dumps({"city": city.title(), "style": style, "daily_usd": city_costs}, indent=2)


@mcp.tool()
def calculate_trip_budget(
    flight_cost: float, hotel_cost: float, daily_expense: float,
    num_days: int, total_budget: float
) -> str:
    """Calculate total trip cost and check if it fits within budget."""
    daily_total = daily_expense * num_days
    total = flight_cost + hotel_cost + daily_total
    remaining = total_budget - total

    return json.dumps({
        "breakdown": {"flights": flight_cost, "hotel": hotel_cost,
                      "daily_expenses": daily_total, "total": total},
        "budget": total_budget, "remaining": remaining,
        "feasible": remaining >= 0,
        "verdict": (f"Within budget! ${remaining:.0f} left." if remaining >= 0
                    else f"Over by ${abs(remaining):.0f}.")
    }, indent=2)


# ═══════════════════════════════════════════
# RESOURCES
# ═══════════════════════════════════════════

@mcp.resource("user://travel-preferences")
def get_user_preferences() -> str:
    """User travel preferences and profile."""
    return json.dumps(USER_PREFS, indent=2)


@mcp.resource("data://tokyo-travel-tips")
def get_tokyo_tips() -> str:
    """Local travel tips for Tokyo."""
    return json.dumps(TOKYO_TIPS, indent=2)


# ═══════════════════════════════════════════
# PROMPTS
# ═══════════════════════════════════════════

@mcp.prompt()
def trip_planner_prompt(destination: str, days: int, budget: float) -> str:
    """System prompt for comprehensive trip planning."""
    return f"""You are an expert travel planner. Plan a {days}-day trip to {destination} within ${budget}.

Steps:
1. Read user://travel-preferences for personalization
2. Use check_calendar to find scheduling conflicts
3. Use search_flights for real flight options
4. Use search_hotels matching user hotel type preference
5. Use estimate_daily_expenses for cost estimation
6. Use calculate_trip_budget to verify feasibility
7. Compile a comprehensive, personalized plan

IMPORTANT: Use REAL data from tools. Never make up prices."""


# ═══════════════════════════════════════════
# Run
# ═══════════════════════════════════════════
if __name__ == "__main__":
    mcp.run(transport="stdio")'''

with open('mcp_server/travel_server.py', 'w') as f:
    f.write(_MCP_SERVER_CODE)

MCP_SERVER_PARAMS = StdioServerParameters(
    command=sys.executable,
    args=['mcp_server/travel_server.py'],
    env={**os.environ, "PYTHONPATH": os.pathsep.join(sys.path)},
)

AGENT_URLS = {
    "currency-agent": "http://localhost:5001",
    "finance-agent": "http://localhost:5002",
}

print("✅ All setup complete")
print("✅ Agent files created: a2a_agents/currency_agent.py, a2a_agents/finance_agent.py")
print("✅ MCP server created: mcp_server/travel_server.py")


---
# 🎯 SECTION A: See the Magic
Just run each cell and watch agents discover each other and collaborate!

## Step 1: Start the agents

In [ ]:
print_header("Starting A2A Agents")

start_agent_server("currency-agent", "a2a_agents/currency_agent.py", 5001)
start_agent_server("finance-agent", "a2a_agents/finance_agent.py", 5002)

print("\n📋 Agent Card URLs:")
print("   Currency: http://localhost:5001/.well-known/agent.json")
print("   Finance:  http://localhost:5002/.well-known/agent.json")

## Step 2: Talk to the Currency Agent — a simple converter

In [ ]:
print_header("Currency Agent — Simple Conversion")

queries = ["100 USD to JPY", "500 EUR to GBP", "What is the weather like?"]
for q in queries:
    print(f'\n💬 "{q}"')
    resp = await send_task_to_agent("http://localhost:5001", q)
    print(f"   → {extract_text_from_response(resp)}")

## Step 3: Talk to the Finance Agent — it has an AI brain inside

In [ ]:
print_header("Finance Agent — AI-Powered Budget Analysis")

queries = [
    "Is a 5-day Tokyo trip feasible for $3000 with $820 flights and $950 hotel?",
    "What are the estimated daily costs in Tokyo?",
    "Convert 3000 USD to JPY",
]
for q in queries:
    print(f'\n💬 "{q}"')
    print("─" * 50)
    resp = await send_task_to_agent("http://localhost:5002", q)
    print(f"📥 {extract_text_from_response(resp)}\n")

## Step 4: LLM Orchestrator — automatically picks the right agent

In [ ]:
print_header("LLM Orchestrator — Agent Selection")

# Discover agents
discovered_agents = {}
for name, url in AGENT_URLS.items():
    try:
        card = await discover_agent(url)
        discovered_agents[name] = {"card": card, "url": url}
        print()
    except Exception as e:
        print(f"  ❌ {name}: {e}\n")

# Build registry for LLM
def build_agent_registry_prompt(agents_dict):
    lines = ["You have access to the following A2A agents:\n"]
    for name, info in agents_dict.items():
        card = info["card"]
        lines.append(f"AGENT: {card.name}")
        lines.append(f"  ID: {name}")
        lines.append(f"  URL: {info['url']}")
        lines.append(f"  Description: {card.description}")
        lines.append(f"  Skills:")
        for skill in card.skills:
            lines.append(f"    - {skill.name}: {skill.description}")
            if skill.examples:
                lines.append(f"      Examples: {', '.join(skill.examples[:2])}")
        lines.append("")
    return "\n".join(lines)

registry_prompt = build_agent_registry_prompt(discovered_agents)

ORCHESTRATOR_TOOLS = [{
    "type": "function",
    "function": {
        "name": "delegate_to_agent",
        "description": "Send a task to an A2A agent for processing",
        "parameters": {
            "type": "object",
            "properties": {
                "agent_id": {"type": "string", "enum": list(discovered_agents.keys()), "description": "ID of the agent"},
                "message": {"type": "string", "description": "The task/message to send"},
            },
            "required": ["agent_id", "message"],
        },
    },
}]

async def llm_orchestrator(user_query):
    print(f"💬 User: {user_query}\n")
    messages = [
        {"role": "system", "content": f"""You are an orchestrator that delegates tasks to specialized A2A agents.
{registry_prompt}
Rules:
- Use currency-agent for simple currency conversions
- Use finance-agent for budget analysis, cost estimation, or financial advice
- You can call multiple agents if needed
- After getting results, compile a final response"""},
        {"role": "user", "content": user_query},
    ]
    all_agent_results = []
    for iteration in range(5):
        response = client_openai.chat.completions.create(model=MODEL, messages=messages, tools=ORCHESTRATOR_TOOLS, temperature=0)
        choice = response.choices[0]
        if choice.finish_reason == "tool_calls" and choice.message.tool_calls:
            messages.append(choice.message.model_dump())
            for tc in choice.message.tool_calls:
                args = json.loads(tc.function.arguments)
                agent_id = args["agent_id"]
                agent_message = args["message"]
                print(f"  🤖 LLM delegates to: {agent_id}")
                print(f'     Message: "{agent_message[:80]}..."' if len(agent_message) > 80 else f'     Message: "{agent_message}"')
                agent_url = discovered_agents[agent_id]["url"]
                try:
                    resp = await send_task_to_agent(agent_url, agent_message)
                    result_text = extract_text_from_response(resp)
                    print(f"  📥 Agent response: {result_text[:150]}...")
                    all_agent_results.append({"agent": agent_id, "result": result_text})
                except Exception as e:
                    result_text = f"Error: {e}"
                    print(f"  ❌ {result_text}")
                messages.append({"role": "tool", "tool_call_id": tc.id, "content": result_text})
        elif choice.finish_reason == "stop":
            final = choice.message.content
            print(f"\n═" * 50)
            print(f"📋 Final Orchestrated Response:\n")
            print(final)
            return final, all_agent_results
    return "Max iterations reached", all_agent_results

result, agents_used = await llm_orchestrator("How much is 2000 USD in Japanese Yen?")

### 👇 Now a complex query that needs BOTH agents

In [ ]:
result, agents_used = await llm_orchestrator(
    "I have a budget of 450000 JPY for a Tokyo trip. "
    "First convert that to USD, then tell me if a 5-day trip "
    "with $820 flights and $950 hotel is feasible within that budget."
)
print(f"\n📊 Agents used: {[a['agent'] for a in agents_used]}")

## Step 5: Grand Finale — LLM + MCP Tools + A2A Agents together
The AI uses its own tools (flights, hotels, calendar) AND delegates to agents (finance, currency).

In [ ]:
# Cell A29: The Grand Orchestrator — MCP tools + A2A agents + LLM brain

from mcp.client.stdio import stdio_client, StdioServerParameters
from mcp.client.session import ClientSession

MCP_SERVER_SCRIPT = os.path.abspath("mcp_server/travel_server.py")

async def grand_orchestrator(user_query):
    """
    The complete agentic system:
    - LLM brain decides what to do
    - MCP tools for direct capabilities (flights, hotels, calendar)
    - A2A agents for delegated tasks (finance, currency)
    """
    print_header("GRAND ORCHESTRATOR: LLM + MCP + A2A")
    print(f"\n💬 User: {user_query}\n")

    # ── Step 1: Connect to MCP server ──
    print_step(1, "Connecting to MCP server for tools")
    server_params = StdioServerParameters(
        command=sys.executable,
        args=[MCP_SERVER_SCRIPT],
        env={**os.environ, "PYTHONPATH": os.pathsep.join(sys.path)},
    )

    # FIX: Provide a writable file object for errlog to avoid UnsupportedOperation: fileno error in Colab
    # Redirect stderr of the child process to /dev/null for the mcp stdio_client
    with open(os.devnull, 'w') as devnull_file:
        async with stdio_client(server_params, errlog=devnull_file) as (read, write):
            async with ClientSession(read, write) as mcp_session:
                await mcp_session.initialize()
                print("  ✅ MCP server connected")

                # ── Step 2: Get MCP tools ──
                tools_result = await mcp_session.list_tools()
                mcp_tools = tools_result.tools
                print(f"  🔧 MCP tools: {[t.name for t in mcp_tools]}")

                # Convert MCP tools to OpenAI format
                openai_mcp_tools = []
                for tool in mcp_tools:
                    openai_mcp_tools.append({
                        "type": "function",
                        "function": {
                            "name": tool.name,
                            "description": tool.description or "",
                            "parameters": tool.inputSchema if tool.inputSchema else {
                                "type": "object", "properties": {}
                            },
                        },
                    })

                # ── Step 3: Add A2A delegation tool ──
                a2a_delegation_tool = {
                    "type": "function",
                    "function": {
                        "name": "delegate_to_a2a_agent",
                        "description": (
                            "Delegate a task to a specialized A2A agent. "
                            "Use 'currency-agent' for simple currency conversions. "
                            "Use 'finance-agent' for budget analysis, cost feasibility, "
                            "and financial advice (this agent uses AI internally)."
                        ),
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "agent_id": {
                                    "type": "string",
                                    "enum": ["currency-agent", "finance-agent"],
                                    "description": "Which agent to delegate to",
                                },
                                "message": {
                                    "type": "string",
                                    "description": "The task/question for the agent",
                                },
                            },
                            "required": ["agent_id", "message"],
                        },
                    },
                }

                # Combine all tools
                all_tools = openai_mcp_tools + [a2a_delegation_tool]
                print(f"  🤖 A2A agents: currency-agent, finance-agent")
                print(f"  📊 Total tools available to LLM: {len(all_tools)}")

                # ── Step 4: Read MCP resources for context ──
                print_step(2, "Reading MCP resources for context")
                prefs = await mcp_session.read_resource("user://travel-preferences")
                prefs_text = prefs.contents[0].text if prefs.contents else "{}"

                tips = await mcp_session.read_resource("data://tokyo-travel-tips")
                tips_text = tips.contents[0].text if tips.contents else "{}"
                print("  📄 Loaded: user preferences + Tokyo tips")

                # ── Step 5: Build system prompt ──
                system_message = f"""You are an intelligent travel planning orchestrator.

Youhave TWO types of capabilities:

## 1. MCP Tools (direct — use for search and data):
- search_flights: Search real flights between airports
- search_hotels: Search hotels with pricing
- check_calendar: Check user's calendar for conflicts
- estimate_daily_expenses: Get daily cost estimates for a city
- calculate_trip_budget: Calculate total budget and feasibility

## 2. A2A Agents (delegate — use for specialized analysis):
- currency-agent: Fast currency conversion (e.g., "100 USD to JPY")
- finance-agent: Intelligent budget analysis using AI (complex financial questions)

## Decision Rules:
- Use MCP tools for: flight search, hotel search, calendar checks
- Use currency-agent for: simple currency conversions
- Use finance-agent for: complex budget analysis, financial recommendations
- You CAN combine MCP tools AND A2A agents in the same workflow
- After gathering all data, compile a comprehensive response

## User Preferences:
{prefs_text}

## Destination Tips:
{tips_text}
"""

                messages = [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_query},
                ]

                # ── Step 6: Agentic Loop ──
                print_step(3, "Starting agentic loop (LLM decides tools vs agents)")

                stats = {
                    "iterations": 0,
                    "mcp_tool_calls": 0,
                    "a2a_delegations": 0,
                    "mcp_tools_used": set(),
                    "agents_used": set(),
                }

                for iteration in range(1, 12):
                    stats["iterations"] = iteration
                    print(f"\n  🔄 Iteration {iteration}")
                    print(f"  {'─' * 45}")

                    response = client_openai.chat.completions.create(
                        model=MODEL,
                        messages=messages,
                        tools=all_tools,
                        temperature=0,
                    )
                    choice = response.choices[0]

                    # ─── LLM wants to call tool(s) ───
                    if choice.finish_reason == "tool_calls" and choice.message.tool_calls:
                        messages.append(choice.message.model_dump())

                        for tc in choice.message.tool_calls:
                            func_name = tc.function.name
                            func_args = json.loads(tc.function.arguments)

                            # ═══ CASE A: A2A Agent Delegation ═══
                            if func_name == "delegate_to_a2a_agent":
                                agent_id = func_args["agent_id"]
                                agent_msg = func_args["message"]
                                stats["a2a_delegations"] += 1
                                stats["agents_used"].add(agent_id)

                                print(f"  🤖 A2A DELEGATION → {agent_id}")
                                print(f"     Message: \"{agent_msg[:80]}...\"" if len(agent_msg) > 80 else f"     Message: \"{agent_msg}\"")

                                agent_url = discovered_agents[agent_id]["url"]
                                try:
                                    resp = await send_task_to_agent(agent_url, agent_msg)

                                    result_text = ""
                                    if hasattr(resp, 'result') and hasattr(resp.result, 'history'):
                                        for msg in resp.result.history:
                                            text = extract_text_from_message(msg)
                                            if text:
                                                result_text += text + "\n"
                                    if not result_text:
                                        result_text = str(resp)

                                    print(f"  📥 Agent response ({len(result_text)} chars)")

                                except Exception as e:
                                    result_text = f"Agent error: {e}"
                                    print(f"  ❌ {result_text}")

                                messages.append({
                                    "role": "tool",
                                    "tool_call_id": tc.id,
                                    "content": result_text,
                                })

                            # ═══ CASE B: MCP Tool Call ═══
                            else:
                                stats["mcp_tool_calls"] += 1
                                stats["mcp_tools_used"].add(func_name)

                                print(f"  🔧 MCP TOOL → {func_name}")
                                print(f"     Args: {json.dumps(func_args)[:80]}")

                                try:
                                    mcp_result = await mcp_session.call_tool(func_name, func_args)
                                    result_text = ""
                                    for block in mcp_result.content:
                                        if hasattr(block, "text"):
                                            result_text += block.text
                                    print(f"  📥 Tool result ({len(result_text)} chars)")

                                except Exception as e:
                                    result_text = json.dumps({"error": str(e)})
                                    print(f"  ❌ MCP error: {e}")

                                messages.append({
                                    "role": "tool",
                                    "tool_call_id": tc.id,
                                    "content": result_text,
                                })

                    # ─── LLM is done ───
                    elif choice.finish_reason == "stop":
                        print(f"\n  ✅ LLM finished!")
                        final_answer = choice.message.content or "No response"
                        return final_answer, stats

                    else:
                        if choice.message.content:
                            return choice.message.content, stats
                        messages.append(choice.message.model_dump())

                return "Max iterations reached", stats


print("✅ Grand orchestrator function defined!")
print("   Combines: MCP tools + A2A agents + LLM brain")

In [ ]:
# Cell A30: Test 1 — Simple flight search (should use MCP tools)

print("=" * 60)
print("  TEST 1: Simple flight search → expects MCP tools")
print("=" * 60)

answer, stats = await grand_orchestrator(
    "Search for flights from SFO to Tokyo on 2026-10-01"
)

print(f"\n{'═' * 55}")
print("📋 FINAL ANSWER:")
print(f"{'═' * 55}")
print(answer)

print(f"\n📊 Stats: MCP calls={stats['mcp_tool_calls']}, A2A delegations={stats['a2a_delegations']}")
print(f"   MCP tools: {stats['mcp_tools_used']}")
print(f"   A2A agents: {stats['agents_used']}")

### 👇 Budget analysis — uses A2A agents

In [ ]:
# Cell A31: Test 2 — Budget analysis → expects A2A delegation to finance-agent

print("=" * 60)
print("  TEST 2: Budget analysis → expects A2A finance-agent")
print("=" * 60)

answer, stats = await grand_orchestrator(
    "I have a budget of 450000 JPY. Convert that to USD and tell me "
    "if a 5-day Tokyo trip with $820 flights and $950 hotel is feasible."
)

print(f"\n{'═' * 55}")
print("📋 FINAL ANSWER:")
print(f"{'═' * 55}")
print(answer)

print(f"\n📊 Stats: MCP calls={stats['mcp_tool_calls']}, A2A delegations={stats['a2a_delegations']}")
print(f"   MCP tools: {stats['mcp_tools_used']}")
print(f"   A2A agents: {stats['agents_used']}")

### 👇 The FULL query — MCP tools + A2A agents working together

In [ ]:
# Cell A32: Test 3 — The FULL comprehensive query
# LLM should use BOTH MCP tools AND A2A agents

print("=" * 60)
print("  TEST 3: FULL QUERY — MCP tools + A2A agents together!")
print("=" * 60)

FULL_QUERY = """
Plan me a 5-day trip to Tokyo in October 2026.
My budget is $3000. I'm flying from San Francisco.

Please:
1. Check my calendar for any conflicts in early October 2026
2. Search for flights from SFO to Tokyo around Oct 1
3. Search for boutique hotels in Tokyo for 5 nights
4. Get a professional budget feasibility analysis considering
   flights, hotel, and daily expenses
5. Also tell me how much my $3000 budget is in Japanese Yen
6. Give me a complete plan with all the details
"""

start_time = time.time()
answer, stats = await grand_orchestrator(FULL_QUERY)
total_time = time.time() - start_time

print(f"\n{'═' * 55}")
print("📋 COMPLETE TRAVEL PLAN:")
print(f"{'═' * 55}")
print(answer)

In [ ]:
# Cell A33: Display the execution statistics

print_header("📊 GRAND FINALE — Execution Statistics")

print(f"""
┌─ Execution Summary ─────────────────────────────
│
│  ⏱️  Total time:        {total_time:.1f}s
│  🔄 Iterations:        {stats['iterations']}
│
│  🔧 MCP Tool Calls:    {stats['mcp_tool_calls']}
│     Tools used:        {', '.join(stats['mcp_tools_used']) if stats['mcp_tools_used'] else 'none'}
│
│  🤖 A2A Delegations:   {stats['a2a_delegations']}
│     Agents used:       {', '.join(stats['agents_used']) if stats['agents_used'] else 'none'}
│
│  📊 Total actions:     {stats['mcp_tool_calls'] + stats['a2a_delegations']}
│
└──────────────────────────────────────────────────
""")

In [ ]:
# Cell A36: Grand comparison table across all parts

print_header("🏆 GRAND COMPARISON: The Full Journey")

print("""
┌──────────────────┬────────────┬─────────────┬─────────────┬──────────────┐
│                  │ Part 1     │ Part 2      │ Part 3      │ Part 7       │
│                  │ Raw LLM    │ Kiwi MCP    │ Custom MCP  │ MCP + A2A    │
├──────────────────┼────────────┼─────────────┼─────────────┼──────────────┤
│ Real data        │ ❌         │ ✅ flights  │ ✅ all      │ ✅ all       │
│ MCP tools        │ 0          │ External    │ 5 custom    │ 5 custom     │
│ MCP resources    │ 0          │ 0           │ 2           │ 2            │
│ A2A agents       │ 0          │ 0           │ 0           │ 2 agents     │
│ Calendar check   │ ❌         │ ❌          │ ✅          │ ✅           │
│ Budget analysis  │ ❌         │ ❌          │ ✅ basic    │ ✅ AI-powered│
│ Currency convert │ ❌         │ ❌          │ ❌          │ ✅ via A2A   │
│ Personalized     │ ❌         │ ❌          │ ✅          │ ✅           │
│ Multi-agent      │ ❌         │ ❌          │ ❌          │ ✅           │
├──────────────────┼────────────┼─────────────┼─────────────┼──────────────┤
│ Architecture     │ LLM only   │ LLM + MCP   │ LLM + MCP   │ LLM+MCP+A2A │
└──────────────────┴────────────┴─────────────┴─────────────┴──────────────┘
""")

## Step 6: Expose agents via ngrok — view in A2A Inspector
This lets you see the agents in the [A2A Inspector](https://a2a-inspector-908687846511.us-central1.run.app/) web UI.

In [ ]:
print_header("Exposing Agents via ngrok")

if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTHTOKEN":
    print("❌ NGROK_AUTH_TOKEN not set. Skipping ngrok tunnels.")
    print("   Set it in the API Keys cell at the top to use A2A Inspector.")
else:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    ngrok.kill()

    tunnel1 = ngrok.connect(5001)
    tunnel2 = ngrok.connect(5002)

    print(f"✅ Currency Agent: {tunnel1.public_url}")
    print(f"✅ Finance Agent:  {tunnel2.public_url}")
    print(f"\n🔍 Open A2A Inspector and paste these URLs:")
    print(f"   https://a2a-inspector-908687846511.us-central1.run.app/")

**Try it:** Open the [A2A Inspector](https://a2a-inspector-908687846511.us-central1.run.app/) and paste the ngrok URLs above to see the agent cards and send messages!

---
# 🔬 SECTION B: Under the Hood
Now let's see how each piece works in detail.

## B1: Currency Agent — the code
A simple agent with no LLM. Just regex parsing and math.

In [ ]:
print_header("Currency Agent Code")
with open('a2a_agents/currency_agent.py') as f:
    print(f.read())

## B2: Finance Agent — the code
This agent has an LLM brain and tools inside. From outside, it looks the same as the currency agent.

In [ ]:
print_header("Finance Agent Code")
with open('a2a_agents/finance_agent.py') as f:
    print(f.read())

## B3: Discover agent capabilities via Agent Cards

In [ ]:
# Cell A11: Discover the agent using A2A Client — read its "business card"

print_header("Discovering Agent via Agent Card")

async def discover_agent(base_url):
    async with httpx.AsyncClient() as httpx_client:
        resolver = A2ACardResolver(
            httpx_client=httpx_client,
            base_url=base_url,
        )
        card = await resolver.get_agent_card()

        print(f"📇 Agent Card Discovered!\n")
        print(f"   Name:        {card.name}")
        print(f"   Description: {card.description}")
        print(f"   URL:         {card.url}")
        print(f"   Version:     {card.version}")
        print(f"   Streaming:   {card.capabilities.streaming if card.capabilities else 'N/A'}")

        print(f"\n   🎯 Skills ({len(card.skills)}):")
        for skill in card.skills:
            print(f"      • {skill.name} [{skill.id}]")
            print(f"        {skill.description}")
            if skill.examples:
                print(f"        Examples: {skill.examples}")

        return card

currency_card = await discover_agent("http://localhost:5001")

## B4: Send a task and inspect the raw response

In [ ]:
# Cell A12: Test sending a task to Currency Agent

print_header("Sending Task to Currency Agent")

response = await send_task_to_agent(
    "http://localhost:5001",
    "Convert 3000 USD to JPY"
)

# Corrected line: Use .model_dump_json() for Pydantic objects
print(f"\n📦 Raw response:\n{response.model_dump_json(indent=2)[:600]}")

print(f"\n{'─' * 50}")
print(f"📋 Extracted text:")
print(f"   {extract_text_from_response(response)}")

## B5: Compare the two agents

In [ ]:
# Cell A20: Compare the two agents

print_header("Comparison: Simple Agent vs LLM-Powered Agent")

print("""
┌──────────────────────┬───────────────────────┬────────────────────────┐
│                      │ Currency Agent         │ Finance Agent          │
│                      │ (Port 5001)            │ (Port 5002)            │
├──────────────────────┼───────────────────────┼────────────────────────┤
│ Internal Brain       │ ❌ No LLM              │ ✅ GPT-4o-mini         │
│ Internal Tools       │ Simple regex + math    │ 3 tools via LLM       │
│ Response Style       │ Fixed format           │ Natural language       │
│ Flexibility          │ Rigid parsing          │ Understands context    │
│ Speed                │ ⚡ Very fast            │ 🐢 Slower (LLM call)  │
│ Cost                 │ 💰 Free                │ 💸 API costs           │
├──────────────────────┼───────────────────────┼────────────────────────┤
│ From outside (A2A)   │ Looks the same!        │ Looks the same!        │
│ Agent Card           │ ✅ Standard             │ ✅ Standard             │
│ Message format       │ ✅ Same                 │ ✅ Same                 │
└──────────────────────┴───────────────────────┴────────────────────────┘

💡 KEY INSIGHT: A2A doesn't care what's INSIDE the agent!
   Simple logic, LLM, MCP tools, database — all look the same from outside.
""")

## B6: Architecture — what we built

In [ ]:
# Cell A34: The complete architecture we built

print_header("🏗️ COMPLETE ARCHITECTURE")

print("""
                         ┌─────────────┐
                         │    USER     │
                         │ "Plan trip" │
                         └──────┬──────┘
                                │
                 ┌──────────────▼──────────────┐
                 │     🧠 LLM ORCHESTRATOR     │
                 │        (GPT-4o-mini)         │
                 │                              │
                 │  Reads: MCP Resources        │
                 │    • user preferences        │
                 │    • Tokyo travel tips        │
                 │                              │
                 │  Decides dynamically:         │
                 │    → Use MCP tool? (direct)  │
                 │    → Delegate to A2A agent?  │
                 └───┬─────────────────┬────────┘
                     │                 │
            MCP Tools│                 │A2A Protocol
        (stdio/local)│                 │(HTTP/JSON-RPC)
                     │                 │
         ┌───────────▼──┐    ┌────────▼──────────────┐
         │  MCP Server  │    │    A2A Agents          │
         │              │    │                        │
         │ 🔧 Tools:    │    │ 🤖 currency-agent     │
         │ •flights     │    │    (port 5001)         │
         │ •hotels      │    │    Simple logic        │
         │ •calendar    │    │    No LLM inside       │
         │ •expenses    │    │                        │
         │ •budget calc │    │ 🤖 finance-agent       │
         │              │    │    (port 5002)         │
         │ 📄 Resources:│    │    LLM + tools inside  │
         │ •preferences │    │    AI-powered analysis  │
         │ •travel tips │    │                        │
         └──────────────┘    └────────────────────────┘
""")

## B7: Key takeaways

In [ ]:
# Cell A35: Key takeaways from the entire demo

print_header("🔑 KEY TAKEAWAYS")

print("""
  1️⃣  LLM ALONE is smart but BLIND
     → Hallucinated prices, no real actions
     → Part 1 demonstrated this clearly

  2️⃣  MCP gives an agent TOOLS and CONTEXT
     → "USB-C for AI" — standard tool integration
     → Tools (actions) + Resources (data) + Prompts (templates)
     → Part 2 (Kiwi.com) & Part 3 (custom) showed this

  3️⃣  A2A lets agents DISCOVER and COLLABORATE
     → "HTTP for Agents" — standard agent communication
     → Agent Cards (discovery) + Tasks (work) + Messages (communication)
     → Part 4-6 showed individual and multi-agent scenarios

  4️⃣  MCP and A2A are COMPLEMENTARY
     → MCP = vertical (agent ↔ tools) — WITHIN an agent
     → A2A = horizontal (agent ↔ agent) — BETWEEN agents
     → Part 7 showed them working TOGETHER

  5️⃣  The LLM is the BRAIN that decides
     → When to use MCP tools (direct action)
     → When to delegate to A2A agents (specialized tasks)
     → We wrote ZERO hardcoded orchestration logic

  6️⃣  A2A agents are OPAQUE from outside
     → Currency agent: simple regex (no LLM)
     → Finance agent: full LLM + tools inside
     → From outside, they look identical (same protocol)

  7️⃣  Both are OPEN STANDARDS
     → MCP: by Anthropic (open source)
     → A2A: by Google (open source)
     → Not locked to any vendor
""")

print(f"""
{'═' * 55}
  🎓 ANALOGY: Think of a HOSPITAL

  🏥 Hospital         = Your AI Application
  👨‍⚕️ Each Doctor       = An Agent (A2A)
  🔧 Their instruments = MCP Tools
  📋 Patient records   = MCP Resources
  📇 Hospital directory= Agent Cards
  📝 Referral system   = A2A Tasks
  🧠 Medical knowledge = LLM
{'═' * 55}
""")

# 🎉 Demo Complete!## What we built and demonstrated:| Part | Component | Key Lesson ||------|-----------|------------|| **1** | Raw LLM | LLMs hallucinate without tools || **2** | LLM + Kiwi MCP | External MCP servers give real data || **3** | LLM + Custom MCP | Build your own tools, resources, prompts || **4** | Simple A2A Agent | Agents expose capabilities via Agent Cards || **5** | A2A Agent + LLM | Agents can be smart internally (opaque) || **6** | Multi-Agent Discovery | LLM orchestrator discovers and delegates || **7** | **Grand Finale** | **MCP + A2A + LLM = Complete agentic system** |## The two protocols:- **MCP** = Plug tools/data into an agent (vertical)- **A2A** = Let agents collaborate (horizontal)- **Together** = Scalable, modular AI ecosystem## The LLM's role:- **Brain** that decides what to do- **Zero hardcoded logic** — LLM figures out the workflow- Dynamically chooses between direct tools and agent delegation

In [ ]:
stop_all_agents()
print("\n✅ Demo complete! All agents stopped.")